# Example of cryptostatemachine
Simulate P2P think-lending with microinsurance with Python module *cryptostatemachine*

## Characters
Alice: Has digital camera she rarely uses
Bob: Want digital camera for family bbq next weekend
AZ: Offers novel, exciting microinsurance solutions on the blockchain

## Data structures
*StateMachine*: a directed-graph based Python class implementation of a state-machine with interactions encoded in both the graph structure and ornamentation. As state evolves, crytpographic Bitcoin-like transaction data is exported to a database (currently json file, next step: blockchain)

*SimpleSM*: descendant of StateMachine, with fixed graph structure for simplest P2P thing-lending work flows.

In [55]:
import os
import sys
import ecdsa as ec #Pure Python elliptic curve cryptography
import binascii

In [56]:
# Attach local path where 
module_path = os.path.abspath(os.path.join('PycharmProjects/csm'))

if module_path not in sys.path:
    sys.path.append(module_path)


In [57]:
import cryptostatemachine.cryptostatemachine as csm
reload(csm)

<module 'cryptostatemachine.cryptostatemachine' from '/home/pavel/PycharmProjects/csm/cryptostatemachine/cryptostatemachine.pyc'>

Define the characters

In [92]:
# Issuer
alice_priv = ec.SigningKey.generate()
alice_priv_hex = binascii.b2a_hex(alice_priv.to_string())
alice_pub = alice_priv.get_verifying_key()
alice_pub_hex = binascii.b2a_hex(alice_pub.to_string())
print 'Alice\'s public key:', alice_pub_hex, '\n'

# Interested party
bob_priv = ec.SigningKey.generate()
bob_priv_hex = binascii.b2a_hex(bob_priv.to_string())
bob_pub = bob_priv.get_verifying_key()
bob_pub_hex = binascii.b2a_hex(bob_pub.to_string())
print 'Bob\'s public key:', bob_pub_hex, '\n'

# Microinsurer
az_priv = ec.SigningKey.generate()
az_priv_hex = binascii.b2a_hex(az_priv.to_string())
az_pub = az_priv.get_verifying_key()
az_pub_hex = binascii.b2a_hex(az_pub.to_string())
print 'AZ public key:', az_pub_hex, '\n'

# P2P platform
p2p_priv = ec.SigningKey.generate()
p2p_priv_hex = binascii.b2a_hex(p2p_priv.to_string())
p2p_pub = p2p_priv.get_verifying_key()
p2p_pub_hex = binascii.b2a_hex(p2p_pub.to_string())
#print 'P2P public key:', p2p_pub

# JSON file to hold transaction data for blockchain (mock-up)
db_file = 'p2pthing.json' # Output file for transaction data


Alice's public key: 85b18e309b78e69d167f1c9f6813b0daf8c0b6f15b6836aab0933e02b64fd370937bdb4a008187e4683c1f9c7d3b239a 

Bob's public key: 8734ef558150511a659a86379cc6fbeb48348b84b002cee090246bf7c59ea04b4216df3755ea2553bb020aca8bd191f9 

AZ public key: 07e2b01b2408325cceea9724fb1aac1e903dfca129c033f30fe1486a7c14fbc766c8fd04be2e4f3664654e89f1908dea 



Let's look at what is inside an instance of a simple csm. Note that dictionaries in Python are sorted lexicographically by their key values (this is a good thing to ensure well-definedness).

In [48]:
# Post offer of camera on P2P platform, back-end creates instance of SimpleSM
ssm = csm.SimpleSM(p2p_pub) # Initialize, and track which p2p platform is calling
# Initial graph

print 'Initial Graph Nodes: \n', ssm.graph.nodes(), '\n'

print 'Initial Graph Edges: \n',ssm.graph.edges(), '\n'

# Decoration of vertices:
print 'Initial Node Decoration: \n', ssm.graph['initial'], '\n'

# Hashing
print 'Instance time stame: \n', ssm.time_stamp, '\n'
print 'Initial instance digest: \n', ssm.whole_digest, '\n'


Initial Graph Nodes: 
['returned', 'offered', 'initial', 'transferred', 'insured', 'interested'] 

Initial Graph Edges: 
[('offered', 'insured'), ('offered', 'interested'), ('initial', 'statehashdigest'), ('initial', 'offered'), ('initial', 'toby'), ('transferred', 'returned'), ('interested', 'transferred')] 

Initial Node Decoration: 
{'statehashdigest': '2a51a0eb741b3efd03e5b89152d64f4f1d950c3bbb8c9897d4a5fddaa42105f9', 'offered': {}, 'toby': '04020c7060710fe5ec5311310514268de047e9792d94f763d053b6f5f4a4399daf3e350bb82e00a7df04723840d22dcf6093213dd2f5adcf05bae1353aac93f301'} 

Instance time stame: 
9f762f41a1d8b7fa0ce2f8b2edf14d10d47df34311ac1fcc75ef4b47bc5f4b72 

Initial instance digest: 
2a51a0eb741b3efd03e5b89152d64f4f1d950c3bbb8c9897d4a5fddaa42105f9 



And here are the initial transaction data. Note that the "in_data" are initialized to nonsense.

In [86]:
# Initial transaction (p2p platform only)
print(ssm.trans_data.get_as_dict())
ssm.export_trans_data(db_file)

{'meta_data': {'ver': -1, 'lock_time': -1, 'vin_sz': -1, 'hash': '04dd98fa3942ec34c5d330c9d63c703f361355c334a7eb0ac49ebc1dd7891c62', 'size': -1}, 'out_data': {'scrip_pub_key': -1, 'value': -1}, 'in_data': {'hash': '2a51a0eb741b3efd03e5b89152d64f4f1d950c3bbb8c9897d4a5fddaa42105f9', 'n': -1}}


In [90]:
# Alice offers camera on p2p platform
alice_message = 'Anyone interested in my camera? It\'s a Sony RX100 II'
alice_msg_signed = alice_priv.sign(alice_message)
# Add decoration to 'offered' node
ssm.update_node(node_name = 'offered',
                  sender = alice_pub,
               signature = alice_msg_signed)

print 'Graph structure as string for hashing: \n', ssm.get_graph_str(), '\n'
print 'Instance digest after update: \n', ssm.whole_digest
# Look at transaction data after node update
print 'Transaction data after camera offer placed: \n', ssm.trans_data.get_as_dict()
#Export to data base
ssm.export_trans_data(db_file)


Graph structure as string for hashing: 
{'returned': {}, 'offered': {'insured': {}, 'interested': {}, 'statehashdigest': 'b9904c3dafd378d60747ff74de356b579832b77908c0251cda96e34a318b760f', 'sender': <ecdsa.keys.VerifyingKey instance at 0x7fb789c174d0>, 'signature': 'u\xed\x8d=\x18*ar\xa2R\x0c\x12i\rd\xac\xc8;0\x9e\xe9\xe8`9\x07\xa0\xe3\x90<\x88h\tO\xe4\xf6*\xc1\xf5\xda_=\x93\xc4\x0e D\x082'}, 'initial': {'toby': '04020c7060710fe5ec5311310514268de047e9792d94f763d053b6f5f4a4399daf3e350bb82e00a7df04723840d22dcf6093213dd2f5adcf05bae1353aac93f301', 'offered': {}, 'statehashdigest': '2a51a0eb741b3efd03e5b89152d64f4f1d950c3bbb8c9897d4a5fddaa42105f9'}, 'transferred': {'returned': {}}, 'insured': {}, 'interested': {'transferred': {}}} 

Instance digest after update: 
b9904c3dafd378d60747ff74de356b579832b77908c0251cda96e34a318b760f
Transaction data after camera offer placed: 
{'meta_data': {'ver': -1, 'lock_time': -1, 'vin_sz': -1, 'hash': 'b9904c3dafd378d60747ff74de356b579832b77908c0251cda96e34

In [91]:
# Bob sees the post and clicks to ask about the camera
bob_message = 'Ohh, a Sony RX100 II. I would like to borrow it.'
bob_msg_signed = bob_priv.sign(bob_message)
# Add decoration to 'offered' node
ssm.update_node(node_name = 'interested',
                  sender = bob_pub,
               signature = bob_msg_signed)

print 'Graph structure as string for hashing: \n', ssm.get_graph_str(), '\n'
print 'Instance digest after update: \n', ssm.whole_digest
# Look at transaction data after node update
print 'Transaction data after camera offer placed: \n', ssm.trans_data.get_as_dict()
#Export to data base
ssm.export_trans_data(db_file)


Graph structure as string for hashing: 
{'returned': {}, 'offered': {'insured': {}, 'interested': {}, 'statehashdigest': 'b9904c3dafd378d60747ff74de356b579832b77908c0251cda96e34a318b760f', 'sender': <ecdsa.keys.VerifyingKey instance at 0x7fb789c174d0>, 'signature': 'u\xed\x8d=\x18*ar\xa2R\x0c\x12i\rd\xac\xc8;0\x9e\xe9\xe8`9\x07\xa0\xe3\x90<\x88h\tO\xe4\xf6*\xc1\xf5\xda_=\x93\xc4\x0e D\x082'}, 'initial': {'toby': '04020c7060710fe5ec5311310514268de047e9792d94f763d053b6f5f4a4399daf3e350bb82e00a7df04723840d22dcf6093213dd2f5adcf05bae1353aac93f301', 'offered': {}, 'statehashdigest': '2a51a0eb741b3efd03e5b89152d64f4f1d950c3bbb8c9897d4a5fddaa42105f9'}, 'transferred': {'returned': {}}, 'insured': {}, 'interested': {'sender': <ecdsa.keys.VerifyingKey instance at 0x7fb789c17488>, 'statehashdigest': 'b70501bde9f235a06fedd20b43c395267866c2de1381e4ba713ac58e152a34c3', 'transferred': {}, 'signature': '\xe1\x92S\x81\xbc\xf2\x05\xb1|\xd7\x93\xc4Ln\xf8\x11\x12D\xef\x85\xef\xe6`\x1cge\xdc\xbd\xdd\xfd\x99

In [93]:
# Alice sees that Bob is interested, and applies for microinsurance with AZ

az_message = 'We can insure your Sony RX100 II for xyz under conditions abc'
az_msg_signed = az_priv.sign(az_message)
# Add decoration to 'offered' node
ssm.update_node(node_name = 'insured',
                  sender = az_pub,
               signature = az_msg_signed)

print 'Graph structure as string for hashing: \n', ssm.get_graph_str(), '\n'
print 'Instance digest after update: \n', ssm.whole_digest
# Look at transaction data after node update
print 'Transaction data after camera offer placed: \n', ssm.trans_data.get_as_dict()
#Export to data base
ssm.export_trans_data(db_file)

Graph structure as string for hashing: 
{'returned': {}, 'offered': {'insured': {}, 'interested': {}, 'statehashdigest': 'b9904c3dafd378d60747ff74de356b579832b77908c0251cda96e34a318b760f', 'sender': <ecdsa.keys.VerifyingKey instance at 0x7fb789c174d0>, 'signature': 'u\xed\x8d=\x18*ar\xa2R\x0c\x12i\rd\xac\xc8;0\x9e\xe9\xe8`9\x07\xa0\xe3\x90<\x88h\tO\xe4\xf6*\xc1\xf5\xda_=\x93\xc4\x0e D\x082'}, 'initial': {'toby': '04020c7060710fe5ec5311310514268de047e9792d94f763d053b6f5f4a4399daf3e350bb82e00a7df04723840d22dcf6093213dd2f5adcf05bae1353aac93f301', 'offered': {}, 'statehashdigest': '2a51a0eb741b3efd03e5b89152d64f4f1d950c3bbb8c9897d4a5fddaa42105f9'}, 'transferred': {'returned': {}}, 'insured': {'statehashdigest': '49532826e1eb795999e5c2269586b422cb0890b919e7bfc817d458c46d89825d', 'sender': <ecdsa.keys.VerifyingKey instance at 0x7fb789ba0d88>, 'signature': "\xa5'\xd5\x1e)\xe3\xc9aa\x93\x03z\xa7\xe8\x04\xbc\x8b&\xbb+f\x05/\x00w\xc5i\xd4\xe5L\xcfIfU:\xf3\xce\x99\xd5\xb8\xa6D_\xfa\x11eI\xbe"}, '

In [103]:
# Alice checks that insurance has been offered,
# and verifies that it is AZ that signed the offer

ssm.graph['insured']
check_az_key = ssm.graph['insured']['sender']
print(check_az_key.verify(ssm.graph['insured']['signature'], az_message))


True


In [105]:
# Alice ships the camera to Bob
shipped_message = 'The camera has been shipped via DHL nr 31415926535'
alice_ship_signed = alice_priv.sign(shipped_message)
# Add decoration to 'offered' node
ssm.update_node(node_name = 'transferred',
                  sender = alice_pub,
               signature = alice_ship_signed)

print 'Graph structure as string for hashing: \n', ssm.get_graph_str(), '\n'
print 'Instance digest after update: \n', ssm.whole_digest
# Look at transaction data after node update
print 'Transaction data after camera offer placed: \n', ssm.trans_data.get_as_dict()
#Export to data base
ssm.export_trans_data(db_file)

Graph structure as string for hashing: 
{'returned': {}, 'offered': {'insured': {}, 'interested': {}, 'statehashdigest': 'b9904c3dafd378d60747ff74de356b579832b77908c0251cda96e34a318b760f', 'sender': <ecdsa.keys.VerifyingKey instance at 0x7fb789c174d0>, 'signature': 'u\xed\x8d=\x18*ar\xa2R\x0c\x12i\rd\xac\xc8;0\x9e\xe9\xe8`9\x07\xa0\xe3\x90<\x88h\tO\xe4\xf6*\xc1\xf5\xda_=\x93\xc4\x0e D\x082'}, 'initial': {'toby': '04020c7060710fe5ec5311310514268de047e9792d94f763d053b6f5f4a4399daf3e350bb82e00a7df04723840d22dcf6093213dd2f5adcf05bae1353aac93f301', 'offered': {}, 'statehashdigest': '2a51a0eb741b3efd03e5b89152d64f4f1d950c3bbb8c9897d4a5fddaa42105f9'}, 'transferred': {'sender': <ecdsa.keys.VerifyingKey instance at 0x7fb789c3f8c0>, 'statehashdigest': '49c9b18ad052311a41ca75645b832a6461767b0d1f17b1ef63254ff26123e8b3', 'returned': {}, 'signature': '\xc6\x03\xb5\xe2\x82&$\x92\xc3~\xce\x8dL\xcfn\xad\x0e(\\\x82~ \xccv\xa6\x0f;\xa7\xd0\xa1\xfcnzA@<\x88tJ\xb2\x85[\xe6<\xfdA\x1d\xb0'}, 'insured': {'sta

In [ ]:
# Happy Ending
# Bob returns the camera to Alice
returned_message = 'The camera was great, shipped back via DHL nr 271828'
bob_ship_signed = bob_priv.sign(returned_message)
# Add decoration to 'offered' node
ssm.update_node(node_name = 'transferred',
                  sender = alice_pub,
               signature = alice_ship_signed)

print 'Graph structure as string for hashing: \n', ssm.get_graph_str(), '\n'
print 'Instance digest after update: \n', ssm.whole_digest
# Look at transaction data after node update
print 'Transaction data after camera offer placed: \n', ssm.trans_data.get_as_dict()
#Export to data base
ssm.export_trans_data(db_file)

In [ ]:
# Alternative ending
# Bob doesn't return camera, AZ pays out claim (coming soon . . .)